In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.optim.lr_scheduler as lr_scheduler
%matplotlib tk

In [53]:
# Define the Sphere function
def sphere(x):
    return np.sum(x**2)

# Gradient of the Sphere function (known)
def sphere_gradient(x):
    # noise = np.random.randn(*x.shape)
    return 2 * x # + noise * np.linalg.norm(x)

cond = 1e6
def ellipsoid(x, cond=cond):
    return sum(cond**(np.arange(len(x)) / (len(x) - 1 + 1e-9)) * np.asarray(x)**2)

def ellipsoid_gradient(x, cond=cond):
    return 2 * cond**(np.arange(len(x)) / (len(x) - 1 + 1e-9)) * np.asarray(x)

In [54]:
def evolution_strategy(f, x0, sigma, iterations=100, mu=5, lambda_=10):
    dim = len(x0)
    x = x0

    # Initialize path variable, decay and damping factors for Cumulative LeArning Rate Adaptation (CLARA)
    path = np.zeros(dim)
    c = 0.2
    d = 0.2
    cos_theta = 0

    # Initialize return variables
    candidate_solutions = []
    step_size = []
    path_norm = []
    step_norm = []
    cos_thetas = []

    weights = np.linspace(1, 2, mu)  # Assign higher weight to best individuals
    weights /= weights.sum()  # Normalize weights to sum to 1

    for i in range(iterations):
        # Generate offspring
        pop = np.random.randn(lambda_, dim)

        # Select mu best individuals
        selected = pop[np.argsort([f(x + sigma * ind) for ind in pop])[:mu]]

        # Update current solution
        step = np.sum(selected.T * weights, axis=1)
        x = x + sigma * step
        candidate_solutions.append(x)

        # Update mean estimate of cosine of the angle change between current and mean gradient
        cos_theta = (1 - c) * cos_theta + np.sqrt(c * (2 - c)) * np.dot(step, path) / (np.linalg.norm(path) * np.linalg.norm(step) + 1e-8)
        cos_thetas.append(cos_theta)

        # Update path
        path = (1 - c) * path + np.sqrt(c * (2 - c)) * step
        path_norm.append(np.linalg.norm(path)**2)
        step_norm.append(np.linalg.norm(step)**2)

        # Update step-size
        sigma = sigma * np.exp(d * ((np.linalg.norm(path)**2 / dim) - 1))
        step_size.append(sigma)


        print(f'Iteration {i}: current fitness = {f(x)}')

    return candidate_solutions, step_size, path_norm, step_norm, cos_thetas

In [70]:
def gradient_descent(gradient, x0, lr=0.1, iterations=100):
    """
    Performs gradient descent to minimize a function.

    :param gradient: Function that computes the gradient ∇f(x).
    :param x0: Initial guess (NumPy array).
    :param lr: Learning rate (step size).
    :param iterations: Number of iterations.
    :return: Final optimized value of x.
    """
    x = x0  # Initialize x
    dim = len(x0)  # Get dimension of search space

    # Initialize path variable, decay and damping factors for Cumulative LeArning Rate Adaptation (CLARA)
    path = np.zeros(dim)
    v = np.zeros(dim)
    cos_theta = 0
    c = 0.2
    d = 1e-4

    # Estimate of average of cos(angle) between 2 Gaussian vectors
    # est_angle = average_angle_gaussian_vectors(dim)

    # Initialize return variables
    candidate_solutions = []
    learning_rate = []
    path_norm = []
    gradient_norm = []
    cos_thetas = []

    for i in range(iterations):
        grad = gradient(x)  # Compute gradient
        x = x - lr * grad  # Update step

        # Update mean estimate of cosine of the angle change between current and mean gradient
        cos_theta = (1 - c) * cos_theta + np.sqrt(c * (2 - c)) * np.dot(grad, path) / (np.linalg.norm(path) * np.linalg.norm(grad) + 1e-8)

        # Update path
        path = (1 - c) * path + np.sqrt(c * (2 - c)) * grad
        v = (1 - c) * v + np.sqrt(c * (2 - c)) * grad**2

        # Update learning rate
        # lr = lr * np.exp(d * (np.linalg.norm(path)**2 / (dim + np.linalg.norm(grad)**2) - 1))
        lr = lr * np.exp(d * (np.linalg.norm(path)**2 / dim - 1))

        candidate_solutions.append(x)
        learning_rate.append(lr)
        path_norm.append(np.linalg.norm(path)**2)
        gradient_norm.append(np.linalg.norm(grad)**2)
        cos_thetas.append(cos_theta)

    print('Optimized x: ', x)

    return candidate_solutions, learning_rate, path_norm, gradient_norm, cos_thetas

In [91]:
dim = 2
x0 = np.ones(2)
lr0 = 1e0  # 1e-3
budget = 500

choice = 0

if choice == 0:
    optimizer = gradient_descent  # evolution_strategy
    f = sphere_gradient
    fig_title = 'Vanilla gradient descent'  # '(mu, lambda)-ES'
else:
    optimizer = evolution_strategy
    f = sphere
    fig_title = '(mu, lambda)-ES'

candidate_sol, learning_rates, path_norm, grad_norm, cos_theta = optimizer(f, x0, lr0, iterations=budget)

Optimized x:  [3.0824503e-10 3.0824503e-10]


In [92]:
results = [[np.linalg.norm(x)**2 for x in candidate_sol],
           learning_rates,
           path_norm,
           grad_norm,
           [path_norm[i] / grad_norm[i] for i in range(len(path_norm))],  # grad_norm[i]
           cos_theta
           ]
fig_titles = ['Distance to optimum',
              'Learning rate',
              'Path norm',
              'Gradient/step norm',
              'Path-gradient/step ratio',
              'Angle between path and current grad./step'
              ]
y_labels = [r'$\|x_t\|^2$',
            r'$\alpha_t$',
            r'$\|p_t\|^2$',
            r'$\|g_t\|^2$',
            r'$\|p_t\|^2 / \|g_t\|^2$',
            r'$\cos(\theta)$'
            ]
colors = ['r',
          'g',
          'b',
          'c',
          'm',
          'y'
          ]

fig, axes = plt.subplots(3, 2, figsize=(12, 12))  # 2 rows, 2 columns
fig.suptitle(f'{fig_title} on the sphere, D = {str(len(x0))}')

for i, ax in enumerate(axes.flat):  # Iterate over subplots
    if i == len(results) - 1:
        ax.plot(results[i], color=colors[i])
    else:
        ax.semilogy(results[i], color=colors[i])
    ax.set_title(fig_titles[i])
    ax.set_xlabel('Iterations')
    ax.set_ylabel(y_labels[i])
    ax.grid(True)

# Adjust layout and show
plt.tight_layout()
plt.show()